In [2]:
import pandas as pd
import numpy as np
import duckdb, sqlalchemy

In [3]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [4]:
satisfaction_df = pd.read_csv('passenger_satisfaction.csv')

In [5]:
satisfaction_df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [6]:
satisfaction_df.columns

Index(['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

**Reasoning for Removing Columns**

We decided to remove columns that did not pertain to the inflight aspect of the airline as we suspect that these categories are not something that the airline can necessarily completely control on their own. Our goal is to help airlines pinpoint categories that they can improve on and raise satisfaction ratings overall and we believe that an airline can change the most within their own aircraft and the services they provide within it.  

In [7]:
satisfaction_df.drop(columns=['Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Online boarding', 'Baggage handling', 'Checkin service', 
                      'Departure Delay in Minutes', 'Arrival Delay in Minutes', 'Unnamed: 0'], inplace=True)

satisfaction_df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Food and drink,Seat comfort,Inflight entertainment,On-board service,Leg room service,Inflight service,Cleanliness,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,5,5,5,4,3,5,5,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,1,1,1,1,5,4,1,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,5,5,5,4,3,4,5,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,2,2,2,2,5,4,2,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,4,5,3,3,4,3,3,satisfied


In [8]:
satisfaction_df = satisfaction_df.rename(columns={"Customer Type": "CustomerType", 
                                                  "Flight Distance": "FlightDistance", 
                                                  "Food and drink": "FoodAndDrink", 
                                                  "Seat comfort": "SeatComfort", 
                                                  "Inflight entertainment": "InflightEntertainment", 
                                                  "Leg room service": "LegRoomService", 
                                                  'Inflight wifi service': 'InflightWifi',
                                                  'On-board service': 'OnboardService',
                                                  'Type of Travel': 'TravelType',
                                                  'Inflight service': 'InflightService',
                                                  'satisfaction': 'Satisfaction'})
satisfaction_df.columns


Index(['id', 'Gender', 'CustomerType', 'Age', 'TravelType', 'Class',
       'FlightDistance', 'InflightWifi', 'FoodAndDrink', 'SeatComfort',
       'InflightEntertainment', 'OnboardService', 'LegRoomService',
       'InflightService', 'Cleanliness', 'Satisfaction'],
      dtype='object')

**Reasons for Removing Long Distance Flights**

We wanted to remove short to mid distance flights when considering satisfaction levels because longer distance flights will cause more categories to aggravate customers as they are on the plane longer than usual. This will expose the weaknesses that airlines have within their flights and will help them realize what they need to improve. In addition, the lower the chance that customers will feel less satsifed with certain categories because they are on the plane for a shorter time. By focusing on the longer flights and removing shorter distance flights, we can create better predictions on satisfaction levels of customers.

In [9]:
%sql cleaned_satisfaction_df << SELECT * FROM satisfaction_df\
WHERE FlightDistance >= 3000

cleaned_satisfaction_df.head()

Returning data to local variable cleaned_satisfaction_df


,id,Gender,CustomerType,Age,TravelType,Class,FlightDistance,InflightWifi,FoodAndDrink,SeatComfort,InflightEntertainment,OnboardService,LegRoomService,InflightService,Cleanliness,Satisfaction
0,118319,Female,Loyal Customer,36,Business travel,Business,3347,3,1,1,3,3,3,3,2,neutral or dissatisfied
1,7467,Female,Loyal Customer,45,Business travel,Business,3334,2,4,5,4,4,4,4,5,satisfied
2,68001,Female,Loyal Customer,52,Business travel,Business,3475,2,4,4,4,4,4,4,3,satisfied
3,73302,Male,Loyal Customer,26,Business travel,Business,3960,1,4,4,4,4,2,4,4,satisfied
4,51947,Male,Loyal Customer,45,Business travel,Business,3100,3,1,3,4,4,4,4,1,satisfied


In [10]:
cleaned_satisfaction_df.to_csv('cleaned_satisfaction_data.csv', index=False)